In [1]:
import fitz  # PyMuPDF
import re

# Function to count dates in the "G." section of a PDF
def count_dates_in_g_section(pdf_data):
    # Open the PDF from the binary data
    doc = fitz.open(stream=pdf_data, filetype="pdf")
    text = ""
    # Extract text from each page
    for page in doc:
        text += page.get_text()
    doc.close()
    
    # Find the section starting with "G." and ending before "H." or the end of the document
    section_g = re.search(r'(?<=G\.).+?(?=H\.|\Z)', text, re.DOTALL)
    if not section_g:
        return 0  # If "G." section is not found, return 0

    section_g_text = section_g.group(0)
    # Define a regex pattern for dates (MM/DD/YYYY or MM/DD/YY)
    date_pattern = re.compile(r'\b\d{1,2}/\d{1,2}/\d{2,4}\b')
    # Find all dates in the "G." section
    dates = date_pattern.findall(section_g_text)

    return len(dates)

# Paths to the PDF files
pdf_paths = [
    "Public_Disclosure_Fatality_1377353.pdf",
    "1453180_5_2_23.pdf",
    "Public_Disclosure_1510126_F_4.24.23_v2.pdf",
    "2023_04_13_ID_1446073.pdf",
]

# Dictionary to hold the counts of dates for each PDF
date_counts = {}

# Process each PDF file and count the dates in the "G." section
for pdf_path in pdf_paths:
    # Read the binary data of the PDF file
    with open(pdf_path, "rb") as f:
        pdf_data = f.read()
    # Count the dates in the "G." section
    date_counts[pdf_path] = count_dates_in_g_section(pdf_data)

date_counts


{'Public_Disclosure_Fatality_1377353.pdf': 13,
 '1453180_5_2_23.pdf': 9,
 'Public_Disclosure_1510126_F_4.24.23_v2.pdf': 1,
 '2023_04_13_ID_1446073.pdf': 8}